# Improve CampusX

## Instructions

1. You have to submit your solutions before **6:00pm tomorrow**
2. The problems that I have given(**except the 1st one**) are problems with which we are also dealing with. So, in a way we are asking for your help through this hackathon.
3. To be honest solving even a single question is a big deal in the given time duration. So, please don't get **frustrated** if you are not able to solve even one problem. 
4. A request - **don't quit**. Infact **never quit**.
5. Marking Scheme for this hackathon is as follows
**Total Marks=200**
but, while adding this to your dashboard, your total marks will be divided by a **factor of 10**.
So, let's say you scored 90 in the hackathon, you dashboard score would be 9
6. Top 3 performers will get a **prize**, when we will meet next.Promise:)


In [407]:
import pandas as pd

batch = pd.read_csv("batch.csv")
users = pd.read_csv("users.csv")

batchTask = pd.read_csv("batch_task.csv")

course = pd.read_csv("course.csv")
content = pd.read_csv("content.csv")

tasks = pd.read_csv("tasks.csv")
module = pd.read_csv("module.csv")

evaluation = pd.read_csv("evaluation.csv")
gyan = pd.read_csv("gyan.csv")
myday = pd.read_csv("myday.csv")

linkPreviewDetails = pd.read_csv("link_preview_details.csv")
urlTask = pd.read_csv("url_task.csv")


**Problem 1** : Write a python function which will receive 3 input i.e. batch name, start date and end date and returns the leaderboard(in that date window) of that particular batch with the following pieces of info - Name, Marks(till now)

Note: If no date range is provided, show the leaderboard for the entire duration of the course

**Marks -10**

In [408]:
# Write your code here
def leaderBoard(batchName, startDate = "2019-01-01", endDate = "2030-01-01") :
    
    maskBatchId = batch['batch_name'] == batchName
    batchId = batch[maskBatchId]['batch_id'].values[0]
    
    maskStudentsInBatch = users['batch_id'] == batchId
    students = users[maskStudentsInBatch]
    studentsId = students['user_id']
    
    maskUserIdForEvaluation = evaluation['user_id'].isin(studentsId).tolist()
    maskDateRangeForEvaluation = (evaluation['evaluation_date'] >= startDate) & (evaluation['evaluation_date'] <= endDate)
    evaluationData = evaluation[maskUserIdForEvaluation & maskDateRangeForEvaluation]
    evaluationMarks = evaluationData.groupby('user_id')['score'].sum()
    
    maskGyanUserId = gyan['user_id'].isin(studentsId).tolist()
    maskGyanDateRange = (gyan['submission_date'] >= startDate) & (gyan['submission_date'] <= endDate)
    gyanData = gyan[maskGyanUserId & maskGyanDateRange]
    gyanMarks = gyanData.groupby('user_id')['gyan_id'].count()
    
    myDayUserId = myday['user_id'].isin(studentsId).tolist()
    myDayDateRange = (myday['entry_date'] >= startDate) & (myday['entry_date'] <= endDate)
    myDayData = myday[myDayUserId & myDayDateRange]
    myDayMarks = myDayData.groupby('user_id')['myday_id'].count()
    
    marks = gyanMarks.add(myDayMarks, fill_value = 0)
    marks = marks.add(evaluationMarks, fill_value = 0).sort_values(ascending = False)
    
    marks = marks.to_frame(name = "Score")
    
    newDf = students[['user_id', 'fname', 'lname']].merge(marks, left_on = "user_id", right_on = "user_id")
    newDf.sort_values("Score", ascending = False, inplace = True)
    newDf.reset_index(drop=True, inplace = True)
    return newDf
    
    
leaderBoard("Machine Learning Kolkata 2")

,user_id,fname,lname,Score
0,75,Ritwik,Ghosh,196.5
1,76,Ritik,Verma,193.0
2,90,Ritayan,Dhara,190.5
3,78,Balaka,Biswas,189.0
4,87,Alekhyo,Banerjee,188.5
5,104,Priscila,Tamang Ghising,187.5
6,96,Srijani,Chaudhury,184.0
7,93,Writuparna,Banerjee,183.0
8,68,Tuhin,Mukherjee,182.5
9,77,Rajtilak,Pal,182.0


**Problem 2**: Write a python function that receives 1 input, batch_name and returns the list of top 10 students who require maximum attention from the teacher.

**Note**: I know this is kind of abstract, but truly we are facing this problem. Try to come up with a solution/algorithm. 

**Tip** : Don't just think in terms of marks, also think about things like late submissions, number of submissions, myday etc

**Tricky scenario**: Aisa bhi ho sakta hai koi student bas shuru me acha perform nai kar raha tha, but ab kar raha hai. So consider dates also.

**Marks : 50**

In [409]:
# Evaluation on the basis of :
# 10 times noOfTask performed out of total given
# 1 for gyan and myday each
# 5 for win
# -5 for late
# 10 times total marks obtained

def attentionStudents(batchName) :
    
    maskBatchName = batch['batch_name'] == batchName
    batchData = batch[maskBatchName]
    batchId = batchData['batch_id'].values[0]
    
    userData = users.groupby('batch_id').get_group(batchId)
    studentsId = userData['user_id']
    
    batchTaskMask = batchTask['batch_id'] == batchId
    taskId = batchTask[batchTaskMask]['task_id']
    totalNumberOfTasks = taskId.count()
    fullScore = tasks[tasks['task_id'].isin(taskId)]['total'].sum()
    
    maskUserIdForEvaluation = evaluation['user_id'].isin(studentsId).tolist()
    evaluationData = evaluation[maskUserIdForEvaluation]
    evaluationStudentGroup = evaluationData.groupby('user_id')
    
    noOfTask = evaluationStudentGroup['task_id'].count()
    totalScore = evaluationStudentGroup['score'].sum()
    noOfLateSubmissions = evaluationStudentGroup['late'].sum()
    noOfTimesWinner = evaluationStudentGroup['winner'].sum()
    
    maskGyanUserId = gyan['user_id'].isin(studentsId).tolist()
    gyanData = gyan[maskGyanUserId]
    gyanMarks = gyanData.groupby('user_id')['gyan_id'].count()
    
    myDayUserId = myday['user_id'].isin(studentsId).tolist()
    myDayData = myday[myDayUserId]
    mydayMarks = myDayData.groupby('user_id')['myday_id'].count()

    taskPerformed = noOfTask / totalNumberOfTasks
    score = totalScore / fullScore
    
    marks = gyanMarks.add(mydayMarks, fill_value = 0)
    marks = marks.add((taskPerformed * 10), fill_value = 0)
    marks = marks.add((noOfTimesWinner * 5), fill_value = 0)
    marks = marks.subtract((noOfLateSubmissions * 5), fill_value = 0)
    marks = marks.add((score * 10), fill_value = 0)

    consistentStudentId = marks.sort_values().index[0:10]

    return userData[userData['user_id'].isin(consistentStudentId).tolist()].reset_index(drop = True)

attentionStudents("Machine Learning Kolkata 2")

,user_id,batch_id,fname,lname,email,dp,college,passing_year,branch,github,medium,portfolio,linkedin,youtube
0,70,3,Sagar,Sarkar,sagar1999sarkarcse@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,Techno main salt lake,2022,CSE,NaN,NaN,NaN,NaN,NaN
1,71,3,Soumyajyoti,Karmakar,ksoumyajyoti@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,E,2022,CSE,https://github.com/soumyajyotigit,https://medium.com/@ksoumyajyoti,NaN,https://www.linkedin.com/in/soumyajyoti-karmak...,NaN
2,73,3,Dilshad,Hussain,dilshadhussain576@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,79,3,Srinjoy,Chowdhury,srinjoy35@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,Academy of Technology,2021,CSE,NaN,NaN,NaN,https://www.linkedin.com/in/srinjoy-chowdhury-...,NaN
4,80,3,Rahul,Ghosh,rahulghosh813@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,Academy Of Technology,2021,CSE,NaN,NaN,NaN,https://www.linkedin.com/in/rahul-ghosh-432852...,NaN
5,85,3,Sarthak,Kundu,kundus101@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,Techno Main,2022,EE,NaN,NaN,NaN,NaN,NaN
6,92,3,Koulik,Ghosh,koulikghosh1357@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
7,98,3,Om,Prakash Shaw,omprakash2517@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
8,99,3,Abhiraj,Gope,rajabhi9635@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
9,105,3,GAZAL,GARG,gazalgarg2298@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,Amity University kolkata,2021,CSE,NaN,NaN,NaN,https://www.linkedin.com/in/gazal-garg-073a2719b,NaN


**Problem 3**: Write a python function that will take batch name and a date range as input and outputs the most consistent student in that given daterange and batch

**Note**: Come up with your definition of consistency

**Marks - 20** 

In [410]:
# Write your code here

# Approach : A student is considered consistent if he :-
# (1) Uploads Myday
# (2) Uploads gyan
# (3) Submits his assignment on time (No Late Submission)
#
# To calculate consistency factor : 
# Award 1 Marks for each gyan and myday
# Award 10 Marks bonus for each task submission
# Award bonus of 5 marks for each task win
# Penalty of 5 marks for each late submission
# Student with highest consistency factor is considered to be consistent

def consistentStudent(batchName, startDate, endDate) :
    
    maskBatchName = batch['batch_name'] == batchName
    batchData = batch[maskBatchName]
    batchId = batchData['batch_id'].values[0]
    
    userData = users.groupby('batch_id').get_group(batchId)
    studentsId = userData['user_id']
    
    maskUserIdForEvaluation = evaluation['user_id'].isin(studentsId).tolist()
    maskDateRangeForEvaluation = (evaluation['evaluation_date'] >= startDate) & (evaluation['evaluation_date'] <= endDate)
    evaluationData = evaluation[maskUserIdForEvaluation & maskDateRangeForEvaluation]
    evaluationStudentGroup = evaluationData.groupby('user_id')
    
    noOfTaskPerformed = evaluationStudentGroup['task_id'].count()
    noOfLateSubmissions = evaluationStudentGroup['late'].sum()
    noOfTimesWinner = evaluationStudentGroup['winner'].sum()
    
    maskGyanUserId = gyan['user_id'].isin(studentsId).tolist()
    maskGyanDateRange = (gyan['submission_date'] >= startDate) & (gyan['submission_date'] <= endDate)
    gyanData = gyan[maskGyanUserId & maskGyanDateRange]
    gyanMarks = gyanData.groupby('user_id')['gyan_id'].count()
    
    myDayUserId = myday['user_id'].isin(studentsId).tolist()
    myDayDateRange = (myday['entry_date'] >= startDate) & (myday['entry_date'] <= endDate)
    myDayData = myday[myDayUserId & myDayDateRange]
    mydayMarks = myDayData.groupby('user_id')['myday_id'].count()

    marks = gyanMarks.add(mydayMarks, fill_value = 0)
    marks = marks.add((noOfTask * 10), fill_value = 0)
    marks = marks.add((noOfTimesWinner * 5), fill_value = 0)
    marks = marks.subtract((noOfLateSubmissions * 5), fill_value = 0)

    consistentStudentId = marks.sort_values(ascending = False).index[0]

    return userData[userData['user_id'] == consistentStudentId].reset_index(drop = True)

consistentStudent("Machine Learning Kolkata 2", "2020-01-10", "2020-02-28")

,user_id,batch_id,fname,lname,email,dp,college,passing_year,branch,github,medium,portfolio,linkedin,youtube
0,90,3,Ritayan,Dhara,dhararitayan@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


**Problem 4**: Find the best student across batches

**Marks - 20**

In [411]:
# Approach : For each batch, find normalized marks of each student. After finding normalized marks find student with highest marks among all batches.
# Normalized Marks : 
# (1) For gyan and myday : Calculate for entire batch and normalize it (we normalize because each batch have different start and end period)
# (2) Normalize win and late submission for each batch (again depends on number of tasks, each batch may have different no. of tasks)    
# (3) Find totalMarksObtained/totalMarks for each student in batch.

# Marking factor :
# Normal for myday and gyan
# 10 times for total marks
# 5 times for bonus
# 5 penalty for late submission

batchId = batch['batch_id']
userGroup = users.groupby('batch_id')['user_id']
batchTaskGroup = batchTask.groupby('batch_id')

allStudentsMarks = pd.Series(dtype = float)

for i in batchId :
    
    userId = userGroup.get_group(i)
    
    evaluationData = evaluation[evaluation['user_id'].isin(userId)]
    totalScore = evaluationData.groupby('user_id')['score'].sum()
    noOfLateSubmissions = evaluationData.groupby('user_id')['late'].sum()
    noOfTimesWinner = evaluationData.groupby('user_id')['winner'].sum()
    
    gyanData = gyan[gyan['user_id'].isin(userId)]
    gyanMarks = gyanData.groupby('user_id')['gyan_id'].count()

    mydayData = myday[myday['user_id'].isin(userId)]
    mydayData = mydayData[~mydayData['entry'].isna()]
    mydayMarks = mydayData.groupby('user_id')['myday_id'].count()
    
    taskId = batchTaskGroup.get_group(i)['task_id']
    fullScore = tasks[tasks['task_id'].isin(taskId)]['total'].sum()
    
    normalizedMyDayMarks = (mydayMarks-mydayMarks.min())/(mydayMarks.max()-mydayMarks.min())
    normalizedgyanMarks = (gyanMarks-gyanMarks.min())/(gyanMarks.max()-gyanMarks.min())
    normalizedNoOfLateSubmissions = (noOfLateSubmissions-noOfLateSubmissions.min())/(noOfLateSubmissions.max()-noOfLateSubmissions.min())
    normalizedNoOfTimesWinner = (noOfTimesWinner-noOfTimesWinner.min())/(noOfTimesWinner.max()-noOfTimesWinner.min())
    marksPercentage = totalScore/fullScore

    marks = normalizedgyanMarks.add(normalizedMyDayMarks, fill_value = 0)
    marks = marks.add((normalizedNoOfTimesWinner * 5), fill_value = 0)
    marks = marks.subtract((normalizedNoOfLateSubmissions * 5), fill_value = 0)
    marks = marks.add((marksPercentage * 10), fill_value = 0)
    
    allStudentsMarks = allStudentsMarks.append(marks)

bestStudentIndex = allStudentsMarks.sort_values(ascending = False).index[0]
users[users['user_id'] == bestStudentIndex]

,user_id,batch_id,fname,lname,email,dp,college,passing_year,branch,github,medium,portfolio,linkedin,youtube
79,90,3,Ritayan,Dhara,dhararitayan@gmail.com,https://campus-x-profile-images.s3.ap-south-1....,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


**Problem 5** : Write a function/piece of analysis that can compare the performance of 2 batches on the whole

**Note**: Again Abstract

**Note 2**: Will be judging you a lot on the basis of this.

**Note 3**: You can get an instant internship under us, if can apply the right thought process to this problem

**Marks - 50**

In [412]:
# Approach : For each batch, find normalized marks of each student. After finding normalized marks find average marks for the batch.
# Normalized Marks : 
# (1) For gyan and myday : Calculate for entire batch and normalize it (we normalize because each batch have different start and end period)
# (2) Normalize win and late submission for each batch (again depends on number of tasks, each batch may have different no. of tasks)    
# (3) Find totalMarksObtained/totalMarks for each student in batch.

# Marking factor :
# Normal for myday and gyan
# 10 times for total marks
# 5 times for bonus
# 5 penalty for late submission

# After this : summation of total normalized marks obtained by each student / no. of students

batchId = [1, 3]
userGroup = users.groupby('batch_id')['user_id']
batchTaskGroup = batchTask.groupby('batch_id')

dataFrame = pd.DataFrame({'batch_id':[], 'average_marks':[]})

for i in batchId :
    
    userId = userGroup.get_group(i)
    
    evaluationData = evaluation[evaluation['user_id'].isin(userId)]
    totalScore = evaluationData.groupby('user_id')['score'].sum()
    noOfLateSubmissions = evaluationData.groupby('user_id')['late'].sum()
    noOfTimesWinner = evaluationData.groupby('user_id')['winner'].sum()
    
    gyanData = gyan[gyan['user_id'].isin(userId)]
    gyanMarks = gyanData.groupby('user_id')['gyan_id'].count()

    mydayData = myday[myday['user_id'].isin(userId)]
    mydayData = mydayData[~mydayData['entry'].isna()]
    mydayMarks = mydayData.groupby('user_id')['myday_id'].count()
    
    taskId = batchTaskGroup.get_group(i)['task_id']
    fullScore = tasks[tasks['task_id'].isin(taskId)]['total'].sum()
    
    normalizedMyDayMarks = (mydayMarks-mydayMarks.min())/(mydayMarks.max()-mydayMarks.min())
    normalizedgyanMarks = (gyanMarks-gyanMarks.min())/(gyanMarks.max()-gyanMarks.min())
    normalizedNoOfLateSubmissions = (noOfLateSubmissions-noOfLateSubmissions.min())/(noOfLateSubmissions.max()-noOfLateSubmissions.min())
    normalizedNoOfTimesWinner = (noOfTimesWinner-noOfTimesWinner.min())/(noOfTimesWinner.max()-noOfTimesWinner.min())
    marksPercentage = totalScore/fullScore

    marks = normalizedgyanMarks.add(normalizedMyDayMarks, fill_value = 0)
    marks = marks.add((normalizedNoOfTimesWinner * 5), fill_value = 0)
    marks = marks.subtract((normalizedNoOfLateSubmissions * 5), fill_value = 0)
    marks = marks.add((marksPercentage * 10), fill_value = 0)
    
    totalMarksOfGroup = marks.sum()
    totalStudents = userId.count()
    
    averageGroupMarks = (totalMarksOfGroup / totalStudents)
    
    averageGroupMarks
    dataFrame = dataFrame.append(pd.DataFrame({'batch_id':[i], 'average_marks':[averageGroupMarks]}))
    
    # allStudentsMarks = allStudentsMarks.append(marks)

# bestStudentIndex = allStudentsMarks.sort_values(ascending = False).index[0]
# users[users['user_id'] == bestStudentIndex]

dataFrame.sort_values('average_marks', ascending = False)

,batch_id,average_marks
0,3.0,6.964474
0,1.0,3.145963


**Problem 6** : Surprise us

**Description**: Find something(point/points of analysis) that might surprise us

**Marks - 50**

In [413]:
# Finding if a student is performing above average or not based on his group.

# Based on Question 1, calculating average marks of student in a group
def averageOfGroup(batchId) :
    
    maskStudentsInBatch = users['batch_id'] == batchId
    students = users[maskStudentsInBatch]
    studentsId = students['user_id']
    
    maskUserIdForEvaluation = evaluation['user_id'].isin(studentsId).tolist()
    evaluationData = evaluation[maskUserIdForEvaluation]
    evaluationMarks = evaluationData.groupby('user_id')['score'].sum()
    
    maskGyanUserId = gyan['user_id'].isin(studentsId).tolist()
    gyanData = gyan[maskGyanUserId]
    gyanMarks = gyanData.groupby('user_id')['gyan_id'].count()
    
    myDayUserId = myday['user_id'].isin(studentsId).tolist()
    myDayData = myday[myDayUserId]
    myDayMarks = myDayData.groupby('user_id')['myday_id'].count()
    
    marks = gyanMarks.add(myDayMarks, fill_value = 0)
    marks = marks.add(evaluationMarks, fill_value = 0).sort_values(ascending = False)
    
    totalMarksOfGroup = marks.sum()
    totalStudents = studentsId.count()
    
    return (totalMarksOfGroup/totalStudents)
    

average = averageOfGroup("Machine Learning Kolkata 2")
studentsName = "Ritik"
batchId = users[users['fname'] == studentsName]['batch_id'].values[0]
userId = users[users['fname'] == studentsName]['user_id'].values[0]
average = averageOfGroup(batchId)

evaluationMarks = evaluation[evaluation['user_id'] == userId]['score'].sum()
myDayMarks = myday[myday['user_id'] == userId]['myday_id'].count()
gyanMarks = gyan[gyan['user_id'] == userId]['gyan_id'].count()

total = evaluationMarks + myDayMarks + gyanMarks
if(total > average) :
    print(studentsName, "is performing good.")
else :
    print(studentsName, "need care.")

Ritik is performing good.


/home/ritik/.local/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/home/ritik/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
